In [ ]:
import re
import numpy as np
import logging

logging.basicConfig(level="INFO")
log = logging.getLogger().info

In [ ]:
logging.getLogger().setLevel("WARN")

In [ ]:
coord = tuple[int, int]
def read(path: str) -> list[tuple[coord, coord]]:
    coords = []
    with open(path) as f:
        for line in f:
            log(line.strip())
            xs = list(map(int, re.findall(r"x=([+-]?[0-9]+)", line)))
            ys = list(map(int, re.findall(r"y=([+-]?[0-9]+)", line)))
            coords.append(((xs[0], ys[0]), (xs[1], ys[1])))
    return coords


In [ ]:
example = read("inputs/day-15-example.txt")
data = read("inputs/day-15.txt")

## Part 1

For a given $y_T$, how many locations are blocked?
Do this by recursively merging intervals.

In [ ]:
def reduce_spans(s: list[coord]) -> list[coord]:
    out = []

    # L to R merge
    l = s.pop(0)
    for i in range(len(s)):
        r = s.pop(0)

        # Merge
        if l[0] <= r[1] + 1 and l[1] >= r[0] - 1:
            new = (min(l[0], r[0]), max(l[1], r[1]))
            log(f"yes: {l} {r} -> {new}")
            l = new
        else:
            log(f"no: {l} {r}")
            out.append(l)
            l = r
    out.append(l)
    return out


In [ ]:
def part1(data, yt):
    spans: dict[int, set[tuple[int, int]]] = {}
    for (sx, sy), (bx, by) in data:
        dist = abs(sx - bx) + abs(sy - by)
        for dy in range(-dist, dist + 1):
            y = sy + dy
            if y != yt:
                continue
            if y not in spans:
                spans[y] = set()
            spans[y].update({(sx - (dist - abs(dy)), sx + (dist - abs(dy)))})
    return spans


In [ ]:
l, r = reduce_spans(sorted(part1(example, 10)[10])).pop()
r - l

In [ ]:
l, r = reduce_spans(sorted(part1(data, 2_000_000)[2_000_000])).pop()
r - l

## Part 2

Do the whole thing: Find a gap in the pattern.

In [ ]:
def part2(data, cmin, cmax):
    spans: dict[int, set[tuple[int, int]]] = {}
    for (sx, sy), (bx, by) in data:
        dist = abs(sx - bx) + abs(sy - by)
        log(f"{(sx, sy)} ± {dist}")
        for dy in range(-dist, dist + 1):
            y = sy + dy
            if y < cmin or y > cmax:
                continue
            if y not in spans:
                spans[y] = set()
            ds = dist - abs(dy)
            spans[y].update({(max(cmin, sx - ds), min(cmax, sx + ds))})
    return spans


In [ ]:
spans

In [ ]:
spans = part2(example, 0, 20)
for y in spans:
    s = reduce_spans(sorted(spans[y]))
    if len(s) > 1:
        final_y = y
        final_x = s[0][-1] + 1
        break

final_x, final_y, final_x * 4_000_000 + final_y


In [ ]:
spans = part2(data, 0, 4_000_000)
for y in spans:
    s = reduce_spans(sorted(spans[y]))
    if len(s) > 1:
        final_y = y
        final_x = s[0][-1] + 1
        break

final_x, final_y, final_x * 4_000_000 + final_y
